In [21]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import sklearn
from __future__ import print_function
import librosa
import numpy as np
import matplotlib.pyplot as plt
from librosa import display,core
from dtw import dtw
from numpy import linalg as LA
from math import sqrt
from scipy import stats
from sklearn import svm
from sklearn.svm import SVR
from mlxtend.plotting import plot_decision_regions
import matplotlib.patches as mpatches
from sklearn.model_selection import cross_val_predict, cross_val_score,cross_validate
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
#%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from mlxtend.plotting import plot_confusion_matrix
from collections import Counter
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler

In [22]:
def accuracy(prediction,actual):
    count=0
    l=len(prediction)
    for i in range(l):
        if(prediction[i]==actual[i]):
            count+=1
    return count/l*100

In [26]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import os

def createTrainingData(start, end):
    samples = []
    telangana_dir = 'audio/Telangana'
    coastal_dir = 'audio/Coastal'
    rayalseema_dir = 'audio/Rayalseema'
    
    regions = {
        'Telangana': telangana_dir,
        'Coastal': coastal_dir,
        'Rayalseema':rayalseema_dir
    }

    for region, region_dir in regions.items():
        if region == 'Telangana':
            file_range = range(239, 357)
        elif region == 'Rayalseema':
            file_range = range(116, 215)
        elif region == 'Coastal':
            file_range = range(1, 102)  # Assuming audio files in Coastal are named from '1.wav' to '101.wav'

        for i in file_range:
            ind = 0
            try:
                file_path = os.path.join(region_dir, f"{i}.wav")
                a, s = librosa.load(file_path)
            except:
                ind = 1

            if ind == 0:
                mfcc = librosa.feature.mfcc(y=a, sr=s)
                temp = mfcc.T[1][start:end]
                for frame in range(10, 50):
                    temp = np.concatenate((temp, mfcc.T[frame][start:end]))
                samples.append([temp, region])

    print(len(samples))
    X = [i[0] for i in samples]
    Y = [j[1] for j in samples]
    return X, Y




In [27]:
g = GaussianMixture(2)
scores = cross_val_score(g, data, label, cv=10, scoring='f1_macro')
scores.mean(),scores.std()  

C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_classi

C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\s

(nan, nan)

In [4]:
X_train, X_test, y_train, y_test  = train_test_split(data,label)
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[('Coastal', 89), ('Rayalseema', 89), ('Telangana', 89)]


In [5]:
g = GaussianMixture(2)
g.fit(X_resampled)
pred = g.predict(X_test)

C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder on the unique labels in y_test
label_encoder.fit(y_test)

# Transform y_test to numeric labels
y_test_numeric = label_encoder.transform(y_test)

# Now, you can calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_numeric, pred)
conf_matrix

array([[ 0, 19,  0],
       [ 2, 30,  0],
       [ 1, 28,  0]], dtype=int64)

In [20]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, pred)

# Access the elements of the confusion matrix
tn = conf_matrix[0, 0]  # True negatives
fp = conf_matrix[0, 1]  # False positives
fn = conf_matrix[1, 0]  # False negatives
tp = conf_matrix[1, 1]  # True positives
conf_matrix

ValueError: Mix of label input types (string and number)

In [ ]:
precision = round(tp/(tp+fp),4)
recall = round(tp/(tp+fn),4)
fmeasure = round(2*((precision*recall)/(precision+recall)),4)
rej = round(tn/(tn+fp),4)
acc = round((tn+tp)/(tn+tp+fp+fn),4)

In [ ]:
print(precision,recall,fmeasure,rej,acc)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Assuming 'y_test' and 'pred' contain your actual labels and predicted labels
# Convert labels to binary (0 and 1) for one-vs-all ROC AUC curve
y_test_bin = label_binarize(y_test, classes=['Coastal', 'Rayalseema', 'Telangana'])
pred_bin = label_binarize(pred, classes=['Coastal', 'Rayalseema', 'Telangana'])

n_classes = y_test_bin.shape[1]

# Initialize variables to store the ROC curve and AUC values
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], pred_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC AUC curves
plt.figure(figsize=(10, 6))
colors = ['b', 'g', 'r']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'ROC curve (area = {roc_auc[i]:0.2f}) for class {i}')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [36]:
import os
import librosa
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Function to extract MFCC features from audio
def extract_features(audio_path):
    try:
        audio, sample_rate = librosa.load(audio_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        return mfccs
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Load and preprocess the audio data
def load_and_preprocess_data(data_dir):
    X = []  # Features
    Y = []  # Labels

    regions = os.listdir(data_dir)
    for region in regions:
        region_dir = os.path.join(data_dir, region)
        if os.path.isdir(region_dir):  # Check if it's a directory
            for filename in os.listdir(region_dir):
                audio_path = os.path.join(region_dir, filename)
                mfccs = extract_features(audio_path)
                if mfccs is not None:
                    X.append(mfccs)
                    Y.append(region)

    return X, Y

# Specify the path to your audio data directory
data_dir = 'audio'

# Load and preprocess the data
X, Y = load_and_preprocess_data(data_dir)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Flatten the features to fit GMM
X_train = [item for sublist in X_train for item in sublist]
X_test = [item for sublist in X_test for item in sublist]

# Train a Gaussian Mixture Model (GMM)
n_components = 3  # You can adjust the number of components as needed
gmm = GaussianMixture(n_components=n_components)
gmm.fit(X_train)

# Predict labels for the test set
predictions = gmm.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


C:\Users\Lavanya Tetakali\AppData\Local\Temp\ipykernel_26984\147644127.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(audio_path, res_type='kaiser_fast')


Error processing audio\.ipynb_checkpoints\Untitled-checkpoint.ipynb: 
Error processing audio\Coastal\1.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\10.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\101.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\11.wav: No module named 'resampy'

This error is lazily reported, having originally occured 

Error processing audio\Coastal\45.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\46.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\47.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Coastal\48.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\libros

Error processing audio\Rayalseema\119.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\120.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\121.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\122.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site

Error processing audio\Rayalseema\158.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\159.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\160.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\161.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site

Error processing audio\Rayalseema\202.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\203.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\204.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Rayalseema\205.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site

Error processing audio\Telangana\264.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\265.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\266.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\267.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-pac

Error processing audio\Telangana\329.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\330.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\331.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-packages\librosa\core\audio.py, line 32, in <module>

----> resampy = lazy.load("resampy")
Error processing audio\Telangana\332.wav: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\Lavanya Tetakali\anaconda3\lib\site-pac

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [35]:
pip install resampy



     ---------------------------------------- 3.1/3.1 MB 2.5 MB/s eta 0:00:00
